# Three Body Problem: Neural Network Training

A simple neural network is trained to learn approximate solutions to the three body problem.<br>
Data is sampled from three body systems with parameters similar to planets in the solar system.

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np
import datetime
import matplotlib.pyplot as plt

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory, TimeHistory
from tf_utils import plot_loss_hist, EpochLoss, TimeHistory
from tf_utils import Identity

from orbital_element import OrbitalElementToConfig, ConfigToOrbitalElement, MeanToTrueAnomaly, G_
from orbital_element import make_model_elt_to_cfg, make_model_cfg_to_elt

from jacobi import CartesianToJacobi, JacobiToCartesian

from g3b_data import make_traj_g3b, make_data_g3b, make_datasets_g3b, traj_to_batch
from g3b_data import make_datasets_solar, make_datasets_hard
from g3b_data import combine_datasets_g3b, combine_datasets_solar
from g3b_plot import plot_orbit_q, plot_orbit_v, plot_orbit_a, plot_orbit_energy, plot_orbit_element
from g3b import KineticEnergy_G3B, PotentialEnergy_G3B, Momentum_G3B, AngularMomentum_G3B
from g3b import VectorError, EnergyError
from g3b import Motion_G3B, make_physics_model_g3b
from g3b import fit_model
from g3b_model_math import make_position_model_g3b_math, make_model_g3b_math
from g3b_model_nn import make_position_model_g3b_nn, make_model_g3b_nn

In [3]:
# Set active GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[1:2], 'GPU')

In [4]:
# Grow GPU memory (must be first operation in TF)
# gpu_grow_memory()

In [5]:
# Lightweight serialization
fname = '../data/g3b/g3b_train.pickle'
vartbl = load_vartbl(fname)

In [6]:
# Description of datasets to be loaded
n_years = 100
sample_freq = 10
traj_size = n_years * sample_freq + 1

In [7]:
# Configuration for loading data sets
# num_data_sets = 50
num_data_sets = 5
batch_size = 64
num_gpus = 1
full_batch_size = num_gpus * batch_size
seed0 = 42

In [8]:
# Build combined solar data sets
ds_trn, ds_val, ds_tst = combine_datasets_solar(num_data_sets=num_data_sets, batch_size=batch_size, seed0=seed0)

Loaded data from ../data/g3b/398004947.pickle.


W0820 11:01:58.203566 140426489472832 deprecation.py:323] From /home/michael/anaconda3/envs/nbody/lib/python3.7/site-packages/tensorflow/python/data/util/random_seed.py:58: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Loaded data from ../data/g3b/1492681748.pickle.
Loaded data from ../data/g3b/3981900377.pickle.
Loaded data from ../data/g3b/1029057319.pickle.
Loaded data from ../data/g3b/1075961698.pickle.



### Create the Math Model as a Benchmark

In [9]:
model_math = make_model_g3b_math(traj_size=traj_size)

In [10]:
optimizer = keras.optimizers.Adam(learning_rate=0.0)

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss'),
        'a': VectorError(regularizer=1.0, name='a_loss'),
        'q0_rec': VectorError(name='q0_loss'),
        'v0_rec': VectorError(name='v0_loss'),
        'H': EnergyError(name='H_loss'),
        'P': VectorError(name='P_loss', regularizer=1.0),
        'L': VectorError(name='L_loss'),
       }

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0,
                'a': 1.0,
                'q0_rec': 1.0,
                'v0_rec': 1.0,
                'H': 1.0,
                'P': 1.0,
                'L': 1.0}

In [11]:
# Compile the full mathematical model
model_math.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

### Train the Neural Network Model

In [45]:
# Configuration for neural network model architecture
hidden_sizes = [64, 16]
skip_layers = True
traj_size = 1001
batch_size = 64

# Training configuration
activity_reg = 1.0E0
learning_rate = 1.0E-4

In [46]:
# Build neural network model
model_nn = make_model_g3b_nn(hidden_sizes=hidden_sizes, skip_layers=skip_layers, 
                             traj_size=traj_size, batch_size=batch_size)

In [47]:
# model_nn.summary()

In [48]:
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss'),
        'a': VectorError(regularizer=1.0E-2, name='a_loss'),
        'q0_rec': VectorError(name='q0_loss'),
        'v0_rec': VectorError(name='v0_loss'),
        'H': EnergyError(name='H_loss'),
        'P': VectorError(name='P_loss', regularizer=1.0E-4),
        'L': VectorError(name='L_loss'),
       }

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0,
                'a': 1.0,
                'q0_rec': 1.0E4,
                'v0_rec': 1.0E4,
                'H': 1.0,
                'P': 1.0,
                'L': 1.0}

In [49]:
# Compile the NN model
model_nn.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [50]:
# Evaluate the NN model on the validation data
model_nn.evaluate(ds_val)

155/155 [==============================] - 11s 68ms/step - loss: 0.0151 - q_loss: 0.0049 - v_loss: 0.0053 - a_loss: 0.0049 - q0_rec_loss: 3.2634e-14 - v0_rec_loss: 3.2553e-14 - H_loss: 2.4143e-08 - P_loss: 1.1818e-16 - L_loss: 2.0982e-14


[0.015091853274832595,
 0.0048598005,
 0.0053437683,
 0.004888256,
 3.263435e-14,
 3.2553102e-14,
 2.414263e-08,
 1.1817788e-16,
 2.098198e-14]

In [51]:
# Compare this to math model - should be the same before training
# loss = 0.0126
# q_loss = 0.0049
# model_math.evaluate(ds_val)

In [52]:
# Set up training
suffix = '_'.join(str(sz) for sz in hidden_sizes)
model_name = f'model_g3b_nn_{suffix}'
model_h5 = f'../models/g3b/{model_name}.h5'
hist_name = model_name.replace('model_', 'hist_')
epochs = 1
save_freq = 'epoch'

In [54]:
try:
    model_nn.load_weights(model_h5)
    model_nn.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    hist = vartbl[hist_name]
    print(f'Loaded {model_name} from {model_h5}.')
except:
    print(f'Unable to load {model_name} from {model_h5}. Fitting...')
    hist = fit_model(model=model_nn, 
                     ds=ds_trn, 
                     epochs=epochs,
                     loss=loss, 
                     optimizer=optimizer,
                     metrics=metrics,
                     save_freq=save_freq,
                     prev_history = None, 
                     batch_num=0)
    vartbl[hist_name] = hist
    save_vartbl(vartbl, fname)

Unable to load model_g3b_nn_64_16 from ../models/g3b/model_g3b_nn_64_16.h5. Fitting...
779/780 [============================>.] - ETA: 0s - loss: 2.0641 - q_loss: 0.0101 - v_loss: 0.0080 - a_loss: 0.0081 - q0_rec_loss: 9.0926e-05 - v0_rec_loss: 1.1275e-04 - H_loss: 7.8358e-04 - P_loss: 1.1741e-16 - L_loss: 2.2102e-04
Epoch 0001; loss 2.06e+00; elapsed 0:01:14
780/780 [==============================] - 65s 83ms/step - loss: 2.0616 - q_loss: 0.0101 - v_loss: 0.0080 - a_loss: 0.0081 - q0_rec_loss: 9.0813e-05 - v0_rec_loss: 1.1261e-04 - H_loss: 7.8258e-04 - P_loss: 1.1735e-16 - L_loss: 2.2074e-04


In [ ]:
# num_epochs = 50
num_epochs = 1
for i in range(num_epochs):
    ts = datetime.datetime.now()
    st = ts.strftime('%Y-%m-%d %H:%M:%S')
    print(f'*** Training loop {i+1:3} *** - {st}')
    hist = fit_model(model=model_nn, 
                     ds=ds_trn, 
                     epochs=epochs,
                     loss=loss, 
                     optimizer=optimizer,
                     metrics=metrics,
                     save_freq=save_freq,
                     prev_history = hist, 
                     batch_num=i)
    vartbl[hist_name] = hist
    save_vartbl(vartbl, fname)

In [ ]:
# Plot the loss
fig, ax = plot_loss_hist(hist, '[64, 16]')

In [ ]:
# Evaluate the trained model on the training data
model_nn.evaluate(ds_trn)

In [ ]:
# Evaluate the trained model on the test data
model_nn.evaluate(ds_tst)